Цель данного блокнота, научиться предсказывать вероятность просрочки по выданным кредитам.

In [ ]:
import pandas as pd
df=pd.read_excel('Credits.xlsx', engine='openpyxl')

In [ ]:
df.sample(3, random_state=42)

In [ ]:
df.shape

In [ ]:
df['Одобрение кредита'].sum()

In [ ]:
df[df['Сумма выдачи'].notna()]['Сумма выдачи'].count()

In [ ]:
df[df['Дата Выдачи'].notna()]['Тип кредита'].value_counts()

In [ ]:
df[df['Дата Выдачи'].notna()]['Сумма выдачи'].count()

из 46551 заявок было одобрено 17266, но выдано (!) 18224 кредита. Вопрос к качеству данных. Будем ориентироваться на выдачу кредита, как факт одобрения.

In [ ]:
df[(df['Дней просрочки'].notna()) & (df['Дней просрочки']!=0)]['Сумма выдачи'].count()

По всем выданным кредитам было допущено 808 фактов просрочки.

Оставим набор данных только с выданными кредитами определенного типа.

In [ ]:
df=df[(df['Сумма выдачи'].notna()) & (df['Тип кредита']=="Потреб")]
df.shape

# Удалим выбросы и заполним пропуски


In [ ]:
df.info()

In [ ]:
df[df['Среднемесячный подтвержденный доход'].isna()]

In [ ]:
# удалим пропуск с доходом
df=df[df['Среднемесячный подтвержденный доход'].notna()]

In [ ]:
df['Запрошенный срок кредита'].hist()

In [ ]:
df[['Среднемесячный подтвержденный доход',
    'Среднемесячный дополнительный доход', 'Среднемесячные расходы',
       'Среднемесячный доход семьи', 'Сумма выдачи']].hist(
    figsize=(10,10), bins=50);

In [ ]:
df[['Среднемесячный подтвержденный доход', 
    'Среднемесячный дополнительный доход', 'Среднемесячные расходы',
       'Среднемесячный доход семьи', 'Сумма выдачи']].describe()

In [ ]:
df[df['Среднемесячный подтвержденный доход']>500000]

In [ ]:
df[df['Среднемесячный доход семьи']>700000]

In [ ]:
df[df['Сумма выдачи']>1000000]

In [ ]:
df=df[df['Среднемесячный подтвержденный доход']<500000]
df=df[df['Среднемесячный доход семьи']<700000]
df=df[df['Сумма выдачи']<1000000]
df.shape

# Закодируем и заместим пропуски

In [ ]:
from datetime import datetime

df['Дата рождения']=df['Дата рождения'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
df['Возраст']=df['Дата рождения'].apply(lambda x: ((datetime.now()-x).days)/365)

In [ ]:
df.info()

In [ ]:
#нет пропусков
df.loc[df['Образование'].isna(),'Образование']='Среднее специальное'
voc_educ={'Ниже среднего':0, 'Среднее':1, 'Среднее специальное':2, 'Незаконченное высшее':3, 'Высшее':4, 'Второе высшее':5,
         'Ученая степень / МВА':6}
df['Образование код']=df['Образование'].map(voc_educ)

In [ ]:
voc_td={'Без сроkа (постоянная занятость)':7, 'Пенсионер':0, 'Не выбрано':1,
       'Индивидуальный предприниматель':5, 'Срочный kонтраkт':6,
       'Исполнитель по граждансkо-правовому договору':3,
       'Частная праkтиkа (уточните)':4, 'Агент на kомиссионном договоре':2}
df['Работа по трудовому договору код']=df['Работа по трудовому договору'].map(voc_td)

In [ ]:
df.loc[df['Категория должности'].isna(), 'Категория должности']='не указан'
voc_pos={'Рабочий':0, 'Военнослужащий':1, 'Служащий':2, 'Государственный гражданский служащий':3, 'Специалист':4, 
        'Высококвалифицированный специалист':5, 'Руководитель начального звена':6, 
         'Руководитель среднего звена':7, 'Руководитель высшего звена':8, 
         'Владелец предприятия/ген.Директор/Главный бухгалтер':9, 'Нотариус':10, 'Судья':11, 'не указан':-1}
df['Категория должности код']=df['Категория должности'].map(voc_pos)

In [ ]:
df.loc[df['Вид деятельности компании'].isna(),'Вид деятельности компании']='не указан'
voc_vd={}
for i in df['Вид деятельности компании'].value_counts().index:
    voc_vd[i]=df['Вид деятельности компании'].value_counts()[i]
df['Вид деятельности компании код']=df['Вид деятельности компании'].map(voc_vd)

In [ ]:
df.loc[df['Количество сотрудников в компании']==datetime(2017, 11, 30, 0, 0), 'Количество сотрудников в компании']='11-30'
df.loc[df['Количество сотрудников в компании'].isna(), 'Количество сотрудников в компании']='не указан'
voc_qnt={'не указан':-1,'Затрудняюсь ответить':0, 'До 10':1, '11-30':11, '31-50':31, '51-100':51, 'Более 100':100}
df['Количество сотрудников в компании код']=df['Количество сотрудников в компании'].map(voc_qnt)

In [ ]:
df.loc[df['Срок работы в организации'].isna(), 'Срок работы в организации']='не указан'
voc_lt={'не указан':-1, 'более 20 лет':240, 'от 1 года до 3 лет':12, 'от 3 до 5 лет':36,
       'от 5 до 10 лет':60, 'от 10 до 20 лет':120, 'от 6 до 12 мес':6,
       'менее 6 мес':0}
df['Срок работы в организации код']=df['Срок работы в организации'].map(voc_lt)

In [ ]:
df.loc[df['Количество рабочих мест за 3 года'].isna(), 'Количество рабочих мест за 3 года']=-1

In [ ]:
df.loc[df['Семейное положение'].isna(),'Семейное положение']='не указан'
dict_ss={'не указан':-1, 'Холост/не замужем':0, 'Гражданский брак':1, 'Женат/ замужем':2, 'В разводе':3, 'Вдовец/ Вдова':4}
df['Семейное положение код']=df['Семейное положение'].map(dict_ss)

In [ ]:
df.loc[df['Количесво членов семьи'].isna(),'Количесво членов семьи']=-1

In [ ]:
df['Наличие детей'].value_counts()

In [ ]:
df.loc[df['Наличие детей'].isna(),'Наличие детей']='Нет'
df['Наличие детей код']=df['Наличие детей'].apply(lambda x: 0 if x=='Нет' else 1)

In [ ]:
df.loc[df['Сумма просрочки'].isna(), 'Сумма просрочки']=0

In [ ]:
df['Просрочка']=df['Сумма просрочки'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
df['Просрочка'].value_counts()

# Обучение без учителя

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df.columns

In [ ]:
df['Плечо']=df['Запрошенная сумма']/df['Среднемесячный доход семьи']

In [ ]:
df['Плечо'].hist(figsize=(18,4), bins=50);

In [ ]:
df[df['Плечо']>1000]

In [ ]:
df=df[df['Плечо']<1000]

In [ ]:
col=['Плечо','Просрочка', 'Сумма выдачи', 'Процентная ставка', 'Срок', 'Среднемесячный подтвержденный доход']
sns.pairplot(df[col],height=3);

In [ ]:
df[col].corr()

In [ ]:
corr = df.corr()
plt.figure(figsize=(14, 14))
sns.heatmap(corr[(corr >= 0.3) | (corr <= -0.3)],
            cmap="RdBu_r", vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

In [ ]:
plt.figure(figsize=(14,14))
sns.clustermap(df.corr())

In [ ]:
df.groupby(['Семейное положение', 'Просрочка'])['Просрочка'].count().unstack().plot(kind='bar')

In [ ]:
df.groupby([ 'Категория должности', 'Просрочка'])['Просрочка'].count().unstack().plot(kind='bar')

## Кластеризация

In [ ]:
df.columns

In [ ]:
col=['Просрочка', 'Возраст', 'Срок']

In [ ]:
df[col].info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

data = preprocessing.StandardScaler().fit_transform(df[col]) #обратите внимание - только выборка 
data

In [ ]:
from sklearn.cluster import KMeans

#метод локтя
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(data)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Количество кластеров")
plt.ylabel("SSE")
plt.show()

In [ ]:
#метод силуэта
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

for n_cluster in range(2, 10):
    kmeans = KMeans(n_clusters=n_cluster).fit(data)
    label = kmeans.labels_
    sil_coeff = silhouette_score(data, label, metric='euclidean')
    print("Для n_clusters={}, коэффициент силуэта {}".format(n_cluster, sil_coeff))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

kmeans = KMeans(n_clusters=5, init='k-means++', max_iter= 1000, n_init= 10, random_state= 3)

y_kmeans = kmeans.fit_predict(data)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[:,0], data[:,1], data[:,2], 
            c=y_kmeans, cmap='viridis',
            edgecolor='k', s=40, alpha = 0.5)


ax.set_title("Кластер")
ax.set_xlabel(col[0])
ax.set_ylabel(col[1])
ax.set_zlabel(col[2])
ax.dist = 10

ax.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], 
           kmeans.cluster_centers_[:,2], 
           s = 300, c = 'r', marker='*', label = 'Centroid')

plt.autoscale(enable=True, axis='x', tight=True)    

plt.show()

In [ ]:
plt.scatter(data[:, 2], data[:, 1], c = kmeans.labels_, s = 20, cmap = 'summer')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 2], centers[:, 1], c = 'blue', s = 100, alpha = 0.9);
plt.show()

In [ ]:
df_result=df[col].copy()
df_result['kmeans']=kmeans.labels_
df_result.sample(10)

In [ ]:
df_analize=df_result.groupby('kmeans')[col].mean()
df_analize['Count']=df_result.groupby('kmeans')['kmeans'].count()
df_analize

In [ ]:
df_analize[['Возраст', 'Срок', 'Просрочка']].plot.bar()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

x=col[2] #Изменяйте столбцы 
y=col[0]
print(x,y)
sns.lmplot( x=x, y=y, data=df_result, fit_reg=False, hue='kmeans', legend=False)
plt.legend(loc='lower right')
plt.show()

In [ ]:
x=col[1] #Изменяйте столбцы 
y=col[2]
g = sns.lmplot(x=x, y=y, hue="kmeans", col="kmeans",
               data=df_result, height=6, aspect=.4, x_jitter=.1)

In [ ]:
g = sns.lmplot(x=x, y=y, hue="kmeans", col="kmeans",
               data=df_result, col_wrap=2, height=4)

# Машинное обучение

## отбор столбцов

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df2=df[df['Просрочка']==1].copy()

In [ ]:
df2=df2.append(df[df['Просрочка']==0].sample(740))

In [ ]:
df2

In [ ]:
#изменять только названия столбцов!!!
col=['Среднемесячный подтвержденный доход',
       'Среднемесячный дополнительный доход', 'Среднемесячные расходы',
       'Среднемесячный доход семьи',  'Запрошенный срок кредита',
       'Запрошенная сумма', 'Возраст',
       'Образование код', 'Работа по трудовому договору код',
       'Категория должности код', 'Вид деятельности компании код',
       'Количество сотрудников в компании код',
       'Срок работы в организации код', 'Семейное положение код',
       'Наличие детей код', 'Плечо']
Y=df2['Просрочка']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit_transform(df2[col]) #обратите внимание - только выборка 
X

In [ ]:
from sklearn.model_selection import train_test_split
#разделим набор на тренировочный и тестовый
# X- объясняющие переменные 
# Y - то, что предсказываем
# train - тренировочный набор, обучающий набор
# test - спрятанные данные от модели, используем для тестирования 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier #импортирует 
tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(tree.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(tree.score(X_test, y_test)))
print('Важность признака')
for i in zip(col, tree.feature_importances_):
    print("{:.3f}".format(i[1]),'<-',i[0])

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

export_graphviz(tree, out_file="tree.dot", class_names=["0", "1"],
feature_names=col, impurity=False, filled=True)

with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

#feature extraction
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train.astype('int'))

In [ ]:
from sklearn.metrics import accuracy_score
print("Точность обучающая: {:.3f}".format(accuracy_score(y_train, model.predict(X_train))))
print("Точность тестовая: {:.3f}".format(accuracy_score(y_test, model.predict(X_test))))

In [ ]:
from sklearn.metrics import r2_score
print('R2 обучающая', r2_score(model.predict(X_train), y_train))
print('R2 тестовая', r2_score(model.predict(X_test), y_test))

In [ ]:
#матрица количества правильно и ошибочно угаданных классов
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, model.predict(X_test)), columns=[0,1], index=[0,1])

In [ ]:
#так же матрица в процентах и более изящном виде
matrix = confusion_matrix(y_test, model.predict(X_test))
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

#Build the plot
import seaborn as sns
plt.figure(figsize=(7,5))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2)

#Add labels to the plot
class_names = ['0', '1']                 # !!!!!! указать названия классов!
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Предсказанные классы')
plt.ylabel('Истинные классы')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

Для оценки качества работы алгоритма на каждом из классов по отдельности введем метрики precision (точность) и recall (полнота).

**Precision (правильные позитивные / на предсказанные как позитивные) = TP / TP + FP**

**Sensitivity aka Recall (правильные позитивные / на все настоящие позитивные) = TP / TP + FN**

Хотя точность и полнота являются очень важными метриками, сами по себе они не дадут вам полной картины. Одним из способов подытожить их является F-мера (F-measure), которая представляет собой гармоническое среднее точности и полноты:

**F1=2 x (precision x recall)/(precision + recall)**

Поскольку F1-мера учитывает точность и полноту, то для бинарной классификации несбалансированных данных она может быть лучшей метрикой, чем правильность.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test)))

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), col[idx]])
print("Значимость признака:")
ar_f.sort(reverse=True)
ar_f

In [ ]:
d_first = len(col)
plt.figure(figsize=(8, 5))
plt.title("Значимость признака")
plt.bar(range(d_first), importances[indices[:d_first]], align='center')
plt.xticks(range(d_first), np.array(col)[indices[:d_first]], rotation=90)
plt.xlim([-1, d_first]);

In [ ]:
from sklearn.svm import SVC
svc = SVC().fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, svc.predict(X_test)), columns=[0,1], index=[0,1])

In [ ]:
print(classification_report(y_test, svc.predict(X_test)))

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(C=0.1)
res=logistic.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy: {:.3f}".format(accuracy_score(y_test, logistic.predict(X_test))))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, logistic.predict(X_test))))
print(classification_report(y_test, logistic.predict(X_test)))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', random_state=0, max_iter=1000, 
                    hidden_layer_sizes=[10, 10], 
                    activation = 'relu', early_stopping=True).fit(X_train, y_train)

In [ ]:
print("Accuracy: {:.3f}".format(accuracy_score(y_test, mlp.predict(X_test))))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, mlp.predict(X_test))))
print(classification_report(y_test, mlp.predict(X_test)))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from matplotlib import pyplot
from sklearn.metrics import f1_score

#Настройка параметров оценивания алгоритма
seed = 7
num_folds = 10
n_estimators = 100
scoring = 'accuracy'

models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('LSVC', LinearSVC()))
models.append(('SVC', SVC()))
models.append(('BG', BaggingClassifier(n_estimators=n_estimators)))
models.append(('RF', RandomForestClassifier(n_estimators=n_estimators)))
models.append(('ET', ExtraTreesClassifier(n_estimators=n_estimators)))
models.append(('AB', AdaBoostClassifier(n_estimators=n_estimators, algorithm='SAMME')))
models.append(('GB', GradientBoostingClassifier(n_estimators=n_estimators)))

#Оценивание эффективности выполнения каждого алгоритма
scores = []
names = []
results = []
predictions = []
msg_row = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    names.append(name)
    results.append(cv_results)
    m_fit = model.fit(X_train, y_train)
    m_predict = model.predict(X_test)
    predictions.append(m_predict)
    m_score = model.score(X_test, y_test)
    scores.append(m_score)
    f1=f1_score(y_test, model.predict(X_test))
    msg = "%s: train = %.3f (%.3f) / test = %.3f / f1=%.3f" % (name, cv_results.mean(), cv_results.std(), m_score, f1)
    msg_row.append(msg)
    print(msg)
    
#Диаграмма размаха («ящик с усами»)
fig = pyplot.figure()
fig.suptitle('Сравнение результатов выполнения алгоритмов')
ax = fig.add_subplot(111)
red_square = dict(markerfacecolor='r', marker='s')
pyplot.boxplot(results, flierprops=red_square)
ax.set_xticklabels(names, rotation=45)
pyplot.show()